In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# TODO:
# import csv of data - DONE
# run checks on data for duplicates, invalid dates
# if a date is a weekend or holiday, use busday_offset() to revert to the previous valid business day - DONE
# (e.g., Saturday would be considered Friday for calculations)
# from that adjusted list of dates calculate number of days difference between adjusted NVD and our (adjusted?) published - DONE
# Sums all CVES and CVEs in SLA and calc SLA % - DONE
# export final csv containing all date adjustments and delta - DONE
# Create table to SLA results for calendar year
# Join current month with full calendar year SLA table - DONE

In [3]:
# define SLA month
month = 'Aug'

In [4]:
# Import current month's raw csv
df = pd.read_csv('file://localhost/Users/bonni/OneDrive/Documents/Jupyter/TestStats2.csv')

In [5]:
df

,ID,NVD,Published,Title
0,1,2021-07-28,2021-08-03,Vuln Title 1
1,2,2021-07-28,2021-08-03,Vuln Title 1
2,3,2021-07-28,2021-08-03,Vuln Title 1
3,4,2021-07-28,2021-08-03,Vuln Title 1
4,5,2021-07-29,2021-08-03,Vuln Title 1
5,6,2021-07-29,2021-08-05,Vuln Title 1
6,7,2021-08-05,2021-08-10,Vuln Title 1
7,8,2021-08-06,2021-07-26,Vuln Title 1
8,9,2021-08-06,2021-08-06,Vuln Title 2
9,10,2021-08-06,2021-08-06,Vuln Title 2


In [6]:
df.columns = ['id', 'nvd', 'pub', 'title']

In [7]:
len(df) #number of rows of data set

20

In [8]:
df.dtypes #check is to see if the data types (or dtypes) have been correctly interpreted.

id        int64
nvd      object
pub      object
title    object
dtype: object

In [9]:
# Check for duplicate CVEs
df[df.duplicated(subset = 'id', keep = False)]

,id,nvd,pub,title


In [10]:
#convert dates to datetime64[D] type to be compatible with busday_offset
nvdpub = df['nvd'].values.astype('datetime64[D]')
ourpub = df['pub'].values.astype('datetime64[D]')

In [11]:
# Current list of US holidays
holidays=['2021-01-01', '2021-01-18', '2021-08-02']

In [12]:
# Adjust dates to account for CVEs that are published on weekends or holidays
bdd = np.busdaycalendar(weekmask='1111100', holidays=holidays)
adjnvddates = np.busday_offset(nvdpub, 0, roll='backward', busdaycal=bdd)

In [13]:
# Do I need to adjust our pub dates? 
bdd = np.busdaycalendar(weekmask='1111100', holidays=holidays)
adjourdates = np.busday_offset(ourpub, 0, roll='backward', busdaycal=bdd)

In [14]:
# append adjusted dates to table
df['NVD Adjust'] = adjnvddates
df['Our Adjust'] = adjourdates

In [15]:
# Calculate number of days difference between adjusted NVD date and our adjusted published date minus weekends and holidays
delta = np.busday_count(adjnvddates, adjourdates, busdaycal=bdd)

In [16]:
# apppend Business Day Delta to table

df['BDDelta'] = delta
df.columns = ['ID', 'NVD Publish', 'Our Publish', 'Title', 'NVD Adjust', 'Out Adjust', 'BDDelta']
df

,ID,NVD Publish,Our Publish,Title,NVD Adjust,Out Adjust,BDDelta
0,1,2021-07-28,2021-08-03,Vuln Title 1,2021-07-28,2021-08-03,3
1,2,2021-07-28,2021-08-03,Vuln Title 1,2021-07-28,2021-08-03,3
2,3,2021-07-28,2021-08-03,Vuln Title 1,2021-07-28,2021-08-03,3
3,4,2021-07-28,2021-08-03,Vuln Title 1,2021-07-28,2021-08-03,3
4,5,2021-07-29,2021-08-03,Vuln Title 1,2021-07-29,2021-08-03,2
5,6,2021-07-29,2021-08-05,Vuln Title 1,2021-07-29,2021-08-05,4
6,7,2021-08-05,2021-08-10,Vuln Title 1,2021-08-05,2021-08-10,3
7,8,2021-08-06,2021-07-26,Vuln Title 1,2021-08-06,2021-07-26,-8
8,9,2021-08-06,2021-08-06,Vuln Title 2,2021-08-06,2021-08-06,0
9,10,2021-08-06,2021-08-06,Vuln Title 2,2021-08-06,2021-08-06,0


In [17]:
# Count CVEs within SLA
CVEs_in_SLA = (np.count_nonzero(delta < 4))
print('CVEs in SLA:', CVEs_in_SLA)

CVEs in SLA: 19


In [18]:
Total_CVEs = len(df.index)
print('Total CVEs:', Total_CVEs)

Total CVEs: 20


In [19]:
raw_sla = CVEs_in_SLA/Total_CVEs
SLA = "{:.2%}".format(raw_sla)
print('SLA:', SLA)

SLA: 95.00%


In [20]:
# create a df for the totals and SLA
# initialize list of lists
data = {'Month': [month],
        'CVEs in SLA':  [CVEs_in_SLA],
        'Total CVEs': [Total_CVEs],
        'SLA':  [SLA]
        }

df2 = pd.DataFrame(data, columns = ['Month', 'CVEs in SLA', 'Total CVEs', 'SLA'])
df2


,Month,CVEs in SLA,Total CVEs,SLA
0,Aug,19,20,95.00%


In [21]:
#Import yearly SLA csv
df3 = pd.read_csv('file://localhost/Users/bonni/OneDrive/Documents/Jupyter/2021SLA.csv')

In [22]:
#Append current month to 2021 SLA csv
yearly_sla = pd.concat([df3, df2], axis=0)
yearly_sla

,Month,CVEs in SLA,Total CVEs,SLA
0,Jan,45,45,100.00%
1,Feb,34,34,100.00%
2,Mar,23,23,100.00%
3,Apr,42,42,100.00%
4,May,43,43,100.00%
5,Jun,23,23,100.00%
6,Jul,45,45,100.00%
0,Aug,19,20,95.00%


In [23]:
# Export final table to CSV - edit month in filename
df.to_csv(r'C:\Users\bonni\OneDrive\Documents\Jupyter\MON_SLAraw.csv', index = False, header=True)

#Export SLA stats table to CSV - edit month in filename
yearly_sla.to_csv(r'C:\Users\bonni\OneDrive\Documents\Jupyter\2021SLA.csv', index = False, header=True)